In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')

%cd /content/gdrive/My\ Drive/TESIS/autism-master/

Mounted at /content/gdrive/
/content/gdrive/My Drive/TESIS/autism-master


In [ ]:
%%capture
import os
import sys
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
!{sys.executable} -m pip install https://api.github.com/repos/paris-saclay-cds/ramp-workflow/zipball/master
!{sys.executable} -m pip install scikit-learn seaborn nilearn
from problem import get_cv
from download_data import fetch_fmri_time_series
from problem import get_train_data
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [ ]:
import time
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

from nilearn.connectome import ConnectivityMeasure

def _load_fmri(fmri_filenames):
    """Load time-series extracted from the fMRI using a specific atlas."""
    return np.array([pd.read_csv(subject_filename,
                                 header=None).values
                     for subject_filename in fmri_filenames])

class FeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, kind = 'correlation', atlas= 'basc064'):
        # make a transformer which will load the time series and compute the
        # connectome matrix
        self.kindCM = kind
        self.atlas = 'fmri_'+atlas
        self.transformer_fmri = make_pipeline(
            FunctionTransformer(func=_load_fmri, validate=False),
            ConnectivityMeasure(kind=self.kindCM, vectorize=False))

    def fit(self, X_df, y):
        fmri_filenames = X_df[self.atlas]
        self.transformer_fmri.fit(fmri_filenames, y)
        return self

    def transform(self, X_df):
        fmri_filenames = X_df[self.atlas]
        X_con = self.transformer_fmri.transform(fmri_filenames)
        X_con = pd.DataFrame(X_con)
        X_con = X_df.index
        X_con.columns = ['fmri_{}'.format(i) for i in range(X_con.columns.size)]
        print('ended transform')                        
        # get the anatomical information
        X_anatomy = X_df[[col for col in X_df.columns
                          if col.startswith('anatomy')]]
        X_anatomy = X_anatomy.drop(columns='anatomy_select')
        cols = X_anatomy.columns
        scaler = StandardScaler()
        X_anatomy = scaler.fit_transform(X_anatomy)
        X_anatomy = pd.DataFrame(X_anatomy, index=X_df.index)
        X_anatomy.columns = cols
        # concatenate both matrices
        return pd.concat([X_anatomy, X_con], axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

class Classifier(BaseEstimator):
    def __init__(self, est = 'RF'):
      if est == 'RF':
        self.clf = RandomForestClassifier(n_estimators = 300)
      elif est == 'XGB': 
        self.clf = XGBClassifier()
      self.clf_ripser = make_pipeline(StandardScaler(), self.clf)
    def fit(self, X, y):
        self.clf_ripser.fit(X, y)
        return self
    def predict(self, X):
        return self.clf_ripser.predict(X)
    def predict_proba(self, X):
        return self.clf_ripser.predict_proba(X)

In [ ]:
data_train, labels_train = get_train_data()

In [ ]:
def evaluation(X, y, ext = FeatureExtractor(), cla = Classifier()):
    pipe = make_pipeline(ext, cla)
    cv = get_cv(X, y)
    results = cross_validate(pipe, X, y, scoring=['roc_auc', 'accuracy', 'precision', 'recall'], cv=cv,
                             verbose=1, return_train_score=True,
                             n_jobs=-1)
    
    return results

In [ ]:
atlass =  ['power_2011'] #'',['basc122', 'basc197', 'craddock_scorr_mean', 'power_2011','msdl', 'basc064', 'harvard_oxford_cort_prob_2mm']
mattype = ['tangent', 'correlation']#['partial correlation', 'tangent', 'correlation']
classif = ['XGB', 'RF']

for a in [1]:
  for m in [1]:
    for case in remaining:
      atl, mat, clas = case
      class FeatureExtractor(BaseEstimator, TransformerMixin):
          def __init__(self):
              # make a transformer which will load the time series and compute the
              # connectome matrix
              self.kindCM = mat
              self.atlas = 'fmri_'+atl
              self.transformer_fmri = make_pipeline(
                  FunctionTransformer(func=_load_fmri, validate=False),
                  ConnectivityMeasure(kind=self.kindCM, vectorize=True))
          def fit(self, X_df, y):
              fmri_filenames = X_df[self.atlas]
              self.transformer_fmri.fit(fmri_filenames, y)
              return self
          def transform(self, X_df):
              fmri_filenames = X_df[self.atlas]
              X_con = self.transformer_fmri.transform(fmri_filenames)
              X_con = pd.DataFrame(X_con)
              X_con.index = X_df.index
              X_con.columns = ['fmri_{}'.format(i) for i in range(X_con.columns.size)]
              print('ended transform')                        
              # get the anatomical information
              X_anatomy = X_df[[col for col in X_df.columns if col.startswith('anatomy')]]
              X_anatomy = X_anatomy.drop(columns='anatomy_select')
              cols = X_anatomy.columns
              scaler = StandardScaler()
              X_anatomy = scaler.fit_transform(X_anatomy)
              X_anatomy = pd.DataFrame(X_anatomy, index=X_df.index)
              X_anatomy.columns = cols
              # concatenate both matrices
              return pd.concat([X_anatomy, X_con], axis=1)
      
      class Classifier(BaseEstimator):
          def __init__(self):
            self.est = clas
            if self.est == 'RF':
              self.clf = RandomForestClassifier(n_estimators = 300)
            elif self.est == 'XGB': 
              self.clf = XGBClassifier()
            self.clf_ripser = make_pipeline(StandardScaler(), self.clf)
          def fit(self, X, y):
              self.clf_ripser.fit(X, y)
              return self
          def predict(self, X):
              return self.clf_ripser.predict(X)
          def predict_proba(self, X):
              return self.clf_ripser.predict_proba(X)

      results = evaluation(data_train, labels_train, ext= FeatureExtractor(), cla=Classifier())
      print(f'{atl},{mat},{clas}')
      print("ROC-AUC: {:.3f} +- {:.3f}".format(np.mean(results['test_roc_auc']), np.std(results['test_roc_auc'])))
      print("accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']), np.std(results['test_accuracy'])))
      print("precision: {:.3f} +- {:.3f}".format(np.mean(results['test_precision']), np.std(results['test_precision'])))
      print("recall: {:.3f} +- {:.3f}".format(np.mean(results['test_recall']), np.std(results['test_recall'])))

# MATRIZ

In [ ]:
%%capture
import os
import sys
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
!{sys.executable} -m pip install https://api.github.com/repos/paris-saclay-cds/ramp-workflow/zipball/master
!{sys.executable} -m pip install scikit-learn seaborn nilearn
from problem import get_cv
from download_data import fetch_fmri_time_series
from problem import get_train_data
!pip install ripser
from ripser import ripser
from persim import plot_diagrams
!pip install tensorflow_addons  
!pip install gudhi
import pickle as pkl
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, KFold, ShuffleSplit
from tensorflow import random_uniform_initializer as rui
import gudhi.representations as tda
import tensorflow as tf
import os.path
import itertools
import h5py
import tensorflow_addons  as tfa
import gudhi              as gd
from scipy.sparse           import csgraph
from scipy.io               import loadmat
from scipy.linalg           import eigh
from sklearn.preprocessing  import LabelEncoder, OneHotEncoder
from sklearn.cluster import KMeans
import pandas as pd
import seaborn as sns
# Reduccion dimensionalidad
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
!pip install git+https://github.com/MathieuCarriere/perslay
!pip install git+https://github.com/MathieuCarriere/sklearn-tda
from perslay import PerslayModel
from scipy.stats import beta
!pip install nilearn
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt
!pip install xgboost
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
import time
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

from nilearn.connectome import ConnectivityMeasure
from nilearn.connectome import vec_to_sym_matrix

from sklearn.pipeline import Pipeline
from perslay import PerslayModel
from ripser import ripser
import tensorflow_addons  as tfa
import gudhi.representations as tda
import tensorflow as tf
import gudhi              as gd
from tensorflow import random_uniform_initializer as rui
from sklearn.preprocessing import MinMaxScaler, StandardScaler


def ripser_diagram(paciente, hom_over_p=2, hom_dimension = 1, distm = True):
    diag = ripser(paciente, distance_matrix=distm, coeff = hom_over_p, maxdim=hom_dimension)['dgms']
    return(diag)

def gen_diags_dict(Dg):
  Len = len(Dg)
  diags_dict = {'Rips_dim_0': [],'Rips_dim_1': []}
  ep = 0.01
  for i in range(Len):
    if len(Dg[i][0]) == 0:
      diags_dict['Rips_dim_0'].append(np.array([[1,1+ep],[1.5,1.5+ep], [0.5,0.5+ep]]))
    else:
      diags_dict['Rips_dim_0'].append(np.array(Dg[i][0]))
    if len(Dg[i][1]) == 0:
      diags_dict['Rips_dim_1'].append(np.array([[1,1+ep],[1.5,1.5+ep], [0.5,0.5+ep]]))
    else:
      diags_dict['Rips_dim_1'].append(np.array(Dg[i][1]))
  return diags_dict

def diag_prepro(diags_dict, use_selector = True, use_prominent = True, thresh = 500, use_scaler = True, use_padding = True):
  
  # https://gudhi.inria.fr/python/3.1.0.rc1/representations.html
  # Whole pipeline
  tmp = Pipeline([
          ("Selector",      tda.DiagramSelector(use=use_selector, point_type="finite")),
          ("ProminentPts",  tda.ProminentPoints(use=use_prominent, num_pts=thresh)),
          ("Scaler",        tda.DiagramScaler(use=use_scaler, scalers=[([0,1], MinMaxScaler())])),
          ("Padding",       tda.Padding(use=use_padding)),
                  ])

  prm = {filt: {"ProminentPts__num_pts": min(thresh, max([len(dgm) for dgm in diags_dict[filt]]))} 
        for filt in diags_dict.keys() if max([len(dgm) for dgm in diags_dict[filt]]) > 0}

  # Apply the previous pipeline on the different filtrations.
  diags = []
  for dt in prm.keys():
      param = prm[dt]
      tmp.set_params(**param)
      diags.append(tmp.fit_transform(diags_dict[dt]))

  # For each filtration, concatenate all diagrams in a single array.
  D, npts = [], len(diags[0])
  for dt in range(len(prm.keys())):
      D.append(np.array(np.concatenate([diags[dt][i][np.newaxis,:] for i in range(npts)],axis=0),dtype=np.float32))
  
  return D

def load_architecture_params(weight, len_d, layer = "Image", perm_op = "mean"):
  perslay_parameters = []
  perslay_channel = {}

  perslay_channel["pweight_train"] = True
  perslay_channel["layer_train"]   = True
  perslay_channel["final_model"]   = tf.keras.Sequential([tf.keras.layers.Flatten()])

  if layer == "Landscape":
    perslay_channel["layer"]           = "Landscape"
    perslay_channel["lsample_num"]     = 100
    perslay_channel["lsample_init"]    = rui(0.0, 1.0)

  if layer == "Image":
    perslay_channel["layer"]           = "Image"
    perslay_channel["image_size"]      = (20, 20)
    perslay_channel["image_bnds"]      = ((-.001, 2.001), (-.001, 2.001))
    perslay_channel["lvariance_init"]  = 3.


  if weight == "gmix":
    perslay_channel["pweight"]       = "gmix"
    perslay_channel["pweight_num"]   = 3
    perslay_channel["pweight_init"]  = np.array(np.vstack([np.random.uniform(0.,1.,[2,3]),
                                                          5.*np.ones([2,3])]), dtype=np.float32)
  if weight == "power":
    perslay_channel["pweight"]       = "power"
    perslay_channel["pweight_init"]  = 1.
    perslay_channel["pweight_power"] = 1

  if weight == "grid":
    perslay_channel["pweight"]       = "grid"
    perslay_channel["pweight_size"]  = [20,20]
    perslay_channel["pweight_bnds"]  = ((-.001, 1.001), (-.001, 1.001))
    perslay_channel["pweight_init"]  = rui(1.0, 1.0)
  
  if weight == "None":
    perslay_channel["pweight"]       = None

  perslay_channel["perm_op"] = perm_op
  # mean, max, sum, topk

  if perm_op == "topk":
    perslay_channel["keep"]  = 10

  perslay_parameters = [perslay_channel for _ in range(len_d)]

  return perslay_parameters

def vector_representations(D, perslay_parameters):
  mirrored_strategy = tf.distribute.MirroredStrategy()
  with mirrored_strategy.scope():
      
      # Final rho network is a simple dense layer to the number of labels 
      rho = tf.keras.Sequential([tf.keras.layers.Dense(2, activation="sigmoid", input_shape=(800,))])
      model = PerslayModel(name="PersLay", diagdim=2, perslay_parameters=perslay_parameters, rho=rho)

      # Optimizer is Adam with exponential decay of learning rate and moving average of variables
      lr = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=.01, decay_steps=20, decay_rate=0.5)
      optimizer = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-4)
      optimizer = tfa.optimizers.MovingAverage(optimizer, average_decay=0.9) 

      # Loss is cross-entropy
      loss = tf.keras.losses.CategoricalCrossentropy()

      # Metric is accuracy
      metrics = [tf.keras.metrics.CategoricalAccuracy()]

  vectors = model.compute_representations(D).numpy()
  return vectors

def triag_array(array):
    return(array[np.triu_indices_from(array)])

def gen_ripser_from_connectome(X_connectome, name = 'ripser'):
    X_rips = []
    Len = len(X_connectome)
    for i in range(Len):
        dgm = ripser_diagram(X_connectome[i])
        X_rips.append(dgm)
    X_rips = np.nan_to_num(X_rips)
    		# get the ripser diagram
    diags_dict = gen_diags_dict(X_rips)
    F = np.array([[]]*Len)
    D = diag_prepro(diags_dict, use_selector = True, use_prominent = True, thresh = 40, use_scaler = False, use_padding = True)
    lend = len(D)
    perslay_parameters = load_architecture_params(weight="None", len_d = lend, layer= "Image", perm_op = "sum")
    X_ripser = vector_representations(D, perslay_parameters)
    X_ripser = np.nan_to_num(X_ripser)
    X_ripser = pd.DataFrame(X_ripser)
    return X_ripser


In [ ]:
atlass =  [1] #['basc122', 'basc197', 'craddock_scorr_mean', 'power_2011','msdl', 'basc064', 'harvard_oxford_cort_prob_2mm']
mattype = [1]#['partial correlation', 'tangent', 'correlation']
#classif = ['XGB', 'RF']

#últimos tres acá
remaining = [['basc197',	 'partial correlation',	 'XGB'],
             ['basc197',	 'partial correlation',	 'RF'],
             ['basc197',	 'tangent',	 'XGB']]

for A in atlass:
  for B in mattype:
    for case in remaining:
      atl, mat, clas = case[0], case[1], case[2]
      print(case)
      class FeatureExtractor(BaseEstimator, TransformerMixin):

          def __init__(self):
              # make a transformer which will load the time series and compute the
              # connectome matrix
              self.kindCM = mat
              self.atlas = 'fmri_'+ atl
              self.transformer_fmri = make_pipeline(
                  FunctionTransformer(func=_load_fmri, validate=False),
                  ConnectivityMeasure(kind=self.kindCM, vectorize=False))

          def fit(self, X_df, y):
              fmri_filenames = X_df[self.atlas]
              self.transformer_fmri.fit(fmri_filenames, y)
              return self

          def transform(self, X_df):
              fmri_filenames = X_df[self.atlas]
              X_con = self.transformer_fmri.transform(fmri_filenames)
              max_value = np.amax(np.abs(X_con)) + 0.0001
              for i in range(X_con.shape[0]):
                  X_con[i] = np.sqrt(2*(1-X_con[i]/max_value))
              # get the ripser diagram
              print('began ripser transform')
              X_ripser = gen_ripser_from_connectome(X_con, name = 'ripser')
              X_ripser.index = X_df.index
              X_ripser.columns = ['ripser_{}'.format(i)
                                      for i in range(X_ripser.columns.size)]
              print('ended ripser transform')                        
              # get the anatomical information
              X_anatomy = X_df[[col for col in X_df.columns
                                if col.startswith('anatomy')]]
              X_anatomy = X_anatomy.drop(columns='anatomy_select')
              cols = X_anatomy.columns
              scaler = StandardScaler()
              X_anatomy = scaler.fit_transform(X_anatomy)
              X_anatomy = pd.DataFrame(X_anatomy, index=X_df.index)
              X_anatomy.columns = cols
              
              # concatenate both matrices
              return pd.concat([X_anatomy, X_ripser], axis=1)
      
      class Classifier(BaseEstimator):
          def __init__(self):
            self.est = clas
            if self.est == 'RF':
              self.clf = RandomForestClassifier(n_estimators = 300)
            elif self.est == 'XGB': 
              self.clf = XGBClassifier()
            self.clf_ripser = make_pipeline(StandardScaler(), self.clf)
          def fit(self, X, y):
              self.clf_ripser.fit(X, y)
              return self
          def predict(self, X):
              return self.clf_ripser.predict(X)
          def predict_proba(self, X):
              return self.clf_ripser.predict_proba(X)

      results = evaluation(data_train, labels_train, ext= FeatureExtractor(), cla=Classifier())
      print(f'{atl},{mat},{clas}')
      print("ROC-AUC: {:.3f} +- {:.3f}".format(np.mean(results['test_roc_auc']), np.std(results['test_roc_auc'])))
      print("accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']), np.std(results['test_accuracy'])))
      print("precision: {:.3f} +- {:.3f}".format(np.mean(results['test_precision']), np.std(results['test_precision'])))
      print("recall: {:.3f} +- {:.3f}".format(np.mean(results['test_recall']), np.std(results['test_recall'])))

basc197,tangent,XGB
ROC-AUC: 0.632 +- 0.033
accuracy: 0.585 +- 0.031
precision: 0.560 +- 0.028
recall: 0.617 +- 0.049


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 71.0min finished
